# Visualização dos Resultados de Espectro de Infravermelhos

Os materiais analisados com FTIR foram os seguintes:

- Poliol (componente A do PU)
- Isocianato (componente B do PU)
- PU puro
- GP4
- PP4
- GPP22
- Grafite Natural (para comparação)
- Grafite Expandido
- GNPs

## Bibliotecas Utilizadas

Utilizou-se as bibliotecas `Plotly` para construir os gráficos, `Pandas` e `Numpy` para manipular os dados, e outras bibliotecas auxiliares para abrir os arquivos e gerar visualizações interativas.

In [1]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

import pandas as pd
pd.options.plotting.backend = "plotly"
import numpy as np
from ipywidgets import interact, widgets
from pathlib import Path

from os import listdir
from os.path import isfile, join

## Abertura dos arquivos

Os arquivos foram exportados para `.csv` pelo software do espectrofotômetro.
Aqui eles são abertos em tabelas com colunas de número de onda e transmitância.

In [2]:
p = Path('src/ftir')

filenames = [f for f in listdir(p) if isfile(join(p, f))]
frames = dict()
for filename in filenames:
        filepath = p/filename
        try:
            frames[filepath.name] = pd.read_csv(filepath, skiprows = [0,1,2,3,4])
            frames[filepath.name].columns = ['Wavenumber','%Transmitance']
        except FileNotFoundError:
            pass

## Visualização dos dados crus

A função `plot_ftir` permite visualizar os dados como estão. Alguns dos espectros estão invertidos, pois foram medidos na escala de absorbância.

In [3]:
def plot_ftir(key):
    fig  = frames[key].plot(
        x = 'Wavenumber',
        y = '%Transmitance',
        kind='line'
    )
    fig.update_layout(xaxis_range = [4000,500])
    return fig

interact(plot_ftir,
         key = list(frames.keys())
        )

interactive(children=(Dropdown(description='key', options=('Grafite-p1.csv', 'Grafite_expandido.csv', 'Grafite…

<function __main__.plot_ftir(key)>

A correção desses gráficos é feita aqui.

In [4]:
absorb = [
    'Grafite_expandido.csv',
    'Grafite-p1.csv',
    'Misto-p1.csv',
    'PAni-ATR-Absor-11-s4.csv',
    'Pani-p1.csv',
    'pu-p2.csv',
]
for key in absorb:
    frames[key]['%Transmitance'] *= -1

## Normalização

Os dados foram normalizados pois diferentes medições tiveram nível de sinal diferentes. Assim é possível visualizá-las lado a lado.

In [5]:
def normalize(x):
    return np.interp(x,[x.min(), x.max()],[0,1])

for key, frame in frames.items():
    mask = frame['Wavenumber'] >= 500
    frame.loc[mask, '%Transmitance'] = normalize(frame.loc[mask,'%Transmitance'])

## Construção dos Gráficos

Cada arquivo foi identificado por um nome para aparecer na legenda.

Cada conjunto de materiais foi agregado em um gráfico.

In [6]:
names = {
    'Grafite_natural.csv': 'Grafite<br>Natural',
    'Grafite_expandido.csv': 'Grafite<br>Expandido',
    'Grafite_nanoplaquetas.csv':'GNPs',
    'Grafite-p1.csv': 'PU + 4%<br> Grafite',
    'Misto-p1.csv': 'PU + 2% Grafite<br> + 2% PAni',
    'Pani-p1.csv': 'PU + 4%<br> PAni',
    'Iso-PAni_quente.csv': 'Mistura líquida<br>Isocianato-PAni',
    'Poliol-PAni_quente.csv': 'Mistura líquida<br>Poliol-PAni',
    'Poliol.csv': 'Poliol<br>líquido',
    'Isocianato.csv': 'Isocianato<br>líquido',
    'PAni-ATR-Absor-11-s4.csv': 'PAni',
    'pu-p2.csv': 'PU Puro',
}

In [7]:
fig = go.Figure()
for k,(key, frame) in enumerate([
    ('Grafite_natural.csv',frames['Grafite_natural.csv']),
    ('Grafite_expandido.csv',frames['Grafite_expandido.csv']),
    ('Grafite_nanoplaquetas.csv',frames['Grafite_nanoplaquetas.csv']),
]):
    df = frame
    fig.add_trace(
        go.Scatter(
            x = df['Wavenumber'],
            y = df['%Transmitance'].apply(lambda x: x+k)/2,
            name = names[key],
            visible = True
        )
    )
fig.update_layout(
    xaxis_range = [4000, 500],
    yaxis_range = [-0.2, 1.5],
    xaxis_title_text = 'Número de onda (1/cm)',
    yaxis_title_text = 'Transmitância (u.a.)',
    yaxis = dict(tickvals = [], zeroline = False),
    font_size = 18,
    legend = {
        'traceorder': 'reversed'
    },
    width = 800,
    height = 600,
    margin = {
        't': 60,
        'b': 40,
        'r': 40,
        'l': 40,
        'pad': 3,
    }
)
fig.add_vline(x = 1633, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    x = 1633,
    y = 1,
    yanchor = 'bottom',
    text = 'C-O<br>estiramento',
    showarrow = False,
)
fig.add_annotation(
    yref = 'paper',
    x = 1633,
    y = 0.05,
    bgcolor = 'white',
    yanchor = 'top',
    text = '1633',
    showarrow = False,
)
fig.add_vline(x = 3373, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    x=3373,
    y=1,
    yanchor = 'bottom',
    text = 'O-H<br>estiramento',
    showarrow = False,
)
fig.add_annotation(
    yref = 'paper',
    x = 3373,
    y = 0.05,
    bgcolor = 'white',
    yanchor = 'top',
    text = '3373',
    showarrow = False,
)
fig

In [8]:
fig = go.Figure()
for k,(key, frame) in enumerate([
    ('PAni-ATR-Absor-11-s4.csv',frames['PAni-ATR-Absor-11-s4.csv']),
    ('Poliol-PAni_quente.csv',frames['Poliol-PAni_quente.csv']),
    ('Poliol.csv',frames['Poliol.csv'])
]):
    df = frame
    fig.add_trace(
        go.Scatter(
            x = df['Wavenumber'],
            y = df['%Transmitance'].apply(lambda x: x+k)/2,
            name = names[key],
            visible = True
        )
    )
fig.update_layout(
    xaxis_range = [4000, 500],
    yaxis_range = [-0.2, 1.5],
    xaxis_title_text = 'Número de onda (1/cm)',
    yaxis_title_text = 'Transmitância (u.a.)',
    yaxis = dict(
        tickvals = [],
        zeroline = False
    ),
    font_size = 20,
    legend = {'traceorder': 'reversed'},
    width = 800,
    height = 600,
    margin = {
        't': 60,
        'b': 40,
        'r': 40,
        'l': 40,
        'pad': 3,
    }
)
fig.add_vline(x = 2938, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    x = 2938,
    xshift = 40,
    y = 1,
    yanchor = 'bottom',
    align = 'left',
    text = 'C=O<br>estiramento',
    showarrow = False,
)
fig.add_vline(x = 2845, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    xshift = 30,
    x = 2938,
    y = 0.05,
    bgcolor = 'white',
    yanchor = 'top',
    text = '2938-2845',
    showarrow = False,
)
fig.add_vline(x = 3373, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    x = 3373,
    xshift = -40,
    y = 1,
    align = 'right',
    yanchor = 'bottom',
    text = 'O-H<br>esticamento',
    showarrow = False,
)
fig.add_annotation(
    yref = 'paper',
    x = 3373,
    y = 0.05,
    bgcolor = 'white',
    yanchor = 'top',
    text = '3373',
    showarrow = False,
)
fig

In [9]:
fig = go.Figure()
for k,(key, frame) in enumerate([
    ('PAni-ATR-Absor-11-s4.csv', frames['PAni-ATR-Absor-11-s4.csv']),
    ('Isocianato.csv', frames['Isocianato.csv']),
    ('Grafite_nanoplaquetas.csv', frames['Grafite_nanoplaquetas.csv']),
]):
    df = frame
    fig.add_trace(
        go.Scatter(
            x = df['Wavenumber'],
            y = df['%Transmitance'].apply(lambda x: x+k)/2,
            name = names[key],
            visible = True
        )
    )
fig.update_layout(
    xaxis_range = [4000, 500],
    yaxis_range = [-0.2, 1.6],
    xaxis_title_text = 'Número de onda (1/cm)',
    yaxis_title_text = 'Transmitância (u.a.)',
    yaxis = dict(
        tickvals = [],
        zeroline = False
    ),
    font_size = 20,
    legend = {'traceorder': 'reversed'},
    width = 800,
    height = 600,
    margin = {
        't':60,
        'b':40,
        'r':40,
        'l':40,
        'pad':3
    }
)
fig.add_vline(x = 2248, line_dash = 'dot')
fig.add_annotation(
    yref = 'paper',
    x=2248,
    y=1,
    yanchor = 'bottom',
    text = 'N=C=O<br>estiramento',
    showarrow = False,
)
fig.add_annotation(
    yref = 'paper',
    x = 2248,
    y = 0.05,
    bgcolor = 'white',
    yanchor = 'top',
    text = '2248',
    showarrow = False,
)
fig

In [10]:
fig = go.Figure()
for k,(key, frame) in enumerate([
    ('PAni-ATR-Absor-11-s4.csv',frames['PAni-ATR-Absor-11-s4.csv']),
    ('Pani-p1.csv',frames['Pani-p1.csv']),
    ('Misto-p1.csv',frames['Misto-p1.csv']),
    ('Grafite-p1.csv',frames['Grafite-p1.csv']),
    ('Grafite_nanoplaquetas.csv',frames['Grafite_nanoplaquetas.csv']),
]):
    df = frame
    fig.add_trace(
        go.Scatter(
            x = df['Wavenumber'],
            y = df['%Transmitance'].apply(lambda x: x+k)/2,
            name = names[key],
            visible = True
        )
    )
fig.update_layout(
    xaxis_range = [4000,500],
    yaxis_range = [-0.2,3],
    xaxis_title_text = 'Número de onda (1/cm)',
    yaxis_title_text = 'Transmitância (u.a.)',
    yaxis = dict(tickvals = [], zeroline = False),
    font_size = 20,
    legend={'traceorder': 'reversed'},
    width = 700,
    height = 800,
    margin = {
        't':60,
        'b':40,
        'r':40,
        'l':40,
        'pad':3
    }
)
fig

In [11]:
def add_vibration_annotation(fig, text, position):
    if type(position) == int:
        fig.add_vline(x = position, line_dash = 'dot')
        fig.add_annotation(
            yref = 'paper',
            x=position,
            y=1,
            yanchor = 'bottom',
            text=text,
            showarrow = False,
        )
        fig.add_annotation(
            yref = 'paper',
            x=position,
            y=0.055,
            bgcolor = 'white',
            yanchor = 'top',
            text=str(position),
            showarrow = False,
        )
    elif type(position) == list:
        fig.add_vline(x = position[0], line_dash = 'dot')
        fig.add_annotation(
            yref = 'paper',
            x=(position[1]+position[0])/2,
            y=1,
            yanchor = 'bottom',
            text=text,
            showarrow = False,
        )
        fig.add_vline(x = position[1], line_dash = 'dot')
        fig.add_annotation(
            yref = 'paper',
            x=(position[1]+position[0])/2,
            y=0.05,
            bgcolor = 'white',
            yanchor = 'top',
            text=str(position[1])+'-'+str(position[0]),
            showarrow = False,
        )
    return fig

In [12]:
fig = go.Figure()
for k,(key, frame) in enumerate([
    ('Poliol.csv',frames['Poliol.csv']),
    ('pu-p2.csv',frames['pu-p2.csv']),
    ('Isocianato.csv',frames['Isocianato.csv']),
]):
    df = frame
    fig.add_trace(
        go.Scatter(
            x = df['Wavenumber'],
            y = df['%Transmitance'].apply(lambda x: x+k)/2,
            name = names[key],
            visible = True
        )
    )
fig.update_layout(
    xaxis_range = [4000, 500],
    yaxis_range = [-0.25, 1.5],
    xaxis_title_text = 'Número de onda (1/cm)',
    yaxis_title_text = 'Transmitância (u.a.)',
    #title_text = 'Espectros de Infra-vermelho',
    yaxis = dict(tickvals = [], zeroline = False),
    font_size = 18,
    legend={'traceorder':'reversed'},
    width = 1100,
    height = 800,
    margin = {
        't':120,
        'b':40,
        'r':40,
        'l':40,
        'pad':3
    }
)
fig = add_vibration_annotation(fig, 'N=C=O<br>esticamento', 2248)
fig = add_vibration_annotation(fig, 'O-H<br>esticamento', 3373)
fig = add_vibration_annotation(fig, 'C-N<br>esticamento', 1108)
fig = add_vibration_annotation(fig, 'C-O<br>esticamento', 1722)
fig = add_vibration_annotation(fig, 'H-C-H<br>esticamento<br>simétrico e<br>antissimétrico', [2857,2925])
fig